### Replica 13: Chat | streaming

#### A. Aprendiendo sobre OpenAIChatGenerator

In [9]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

messages = [
    ChatMessage.from_system("Siempre responde en español, sin importar si el input es en otro idioma"),
    ChatMessage.from_user("What's Natural Language Processing? Be brief."),
]

chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")
chat_generator.run(messages=messages)

{'replies': [ChatMessage(content='El Procesamiento de Lenguaje Natural (PLN) es una rama de la inteligencia artificial que se ocupa de la interacción entre las computadoras y el lenguaje humano. Su objetivo es permitir que las máquinas entiendan, interpreten y generen texto o hablados de manera que resulte natural y útil para los usuarios. Esto incluye tareas como el análisis de sentimientos, traducción automática, chatbots y resumen de textos, entre otros.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 86, 'prompt_tokens': 33, 'total_tokens': 119, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]}

#### B. Activando streaming

In [10]:
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

chat_generator = OpenAIChatGenerator(model="gpt-4o-mini", streaming_callback=print_streaming_chunk)
response = chat_generator.run(messages=messages)

El procesamiento de lenguaje natural (PLN) es una rama de la inteligencia artificial que se centra en la interacción entre las computadoras y los humanos a través del lenguaje natural. Su objetivo es permitir que las máquinas comprendan, interpreten y generen texto en lenguaje humano de manera útil y efectiva. Esto incluye tareas como la traducción automática, el análisis de sentimientos y la generación de texto.

#### C. Llamar funciones desde un pipeline de Haystack

In [11]:
from haystack import Pipeline, Document
from haystack.components.preprocessors import DocumentCleaner
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

#! Preparar documentos
documents = [
    Document(content="My name is Jean and I live in Paris."),
    Document(content="My name is Mark and I live in Berlin."),
    Document(content="My name is Giorgio and I live in Rome."),
    Document(content="My name is Marta and I live in Madrid."),
    Document(content="My name is Harry and I live in London."),
]

#! Preparar componentes
document_store = InMemoryDocumentStore()
document_cleaner = DocumentCleaner()
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store=document_store)

#! Construir pipeline
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("cleaner", document_cleaner)
indexing_pipeline.add_component("embedder", document_embedder)
indexing_pipeline.add_component("writer", document_writer)

indexing_pipeline.connect("cleaner", "embedder")
indexing_pipeline.connect("embedder", "writer")

#! Ejecutar pipeline
indexing_pipeline.run({"cleaner": {"documents": documents}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'writer': {'documents_written': 5}}

In [19]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

template = [
    ChatMessage.from_system(
    """
    Answer the questions based on the given context.

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}
    Question: {{ question }}
    Answer:
    """
    )
]

rag_pipe = Pipeline()
rag_pipe.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
rag_pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
rag_pipe.add_component("prompt_builder", ChatPromptBuilder(template=template))
rag_pipe.add_component("llm", OpenAIChatGenerator(model="gpt-4o-mini"))

rag_pipe.connect("embedder.embedding", "retriever.query_embedding")
rag_pipe.connect("retriever", "prompt_builder.documents")
rag_pipe.connect("prompt_builder", "llm.messages")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [20]:
query = "Where does Mark live?"
rag_pipe.run({"embedder": {"text": query}, "prompt_builder": {"question": query}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'llm': {'replies': [ChatMessage(content='Mark lives in Berlin.', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 5, 'prompt_tokens': 94, 'total_tokens': 99, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}})]}}

In [21]:
def rag_pipeline_func(query: str):
    result = rag_pipe.run({"embedder": {"text": query}, "prompt_builder": {"question": query}})

    return {"reply": result["llm"]["replies"][0].content}

rag_pipeline_func("Where does Mark live?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reply': 'Mark lives in Berlin.'}

In [22]:
rag_pipeline_func("Donde vivo yo?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reply': 'No se puede responder a la pregunta ya que no se proporciona información sobre quién eres. Necesitaría saber tu nombre para ofrecerte una respuesta adecuada.'}